In [ ]:
print("toca chambear pepepeeee")

In [28]:
import json 
from collections import deque

class Nodo:
    def __init__(self, data=None):
        self.next = None
        self.data = data
        self.bef = None


class ColaDob:
    def __init__(self, limit=9):
        self.limite = limit
        self.front = None
        self.rear = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def isFull(self):
        return self.size >= self.limite

    def enColarD(self, data):
        nodo =Nodo(data)
        if self.isFull():
            print("Cola llena")

        if self.isEmpty():
            self.front = self.rear = nodo
        else:
            nodo.bef= self.rear
            self.rear.next = nodo
            self.rear=nodo
        self.size += 1
        return data
    

    def desenColarD(self):
        if self.isEmpty():
            print("Cola vacía")
            return None
        nodo = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        else:
            self.front.prev = None
        self.size -= 1
        return nodo.data

    def imprimirColaD(self):
        valor = self.front
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")


    def imprimirColaIz(self):
        valor = self.rear
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.bef
        print("None")
    


    def captarColaS(self, ColaS):
        if ColaS.isEmpty():
            return
        else:
          self.enColarD(ColaS.desenColar())
          self.captarColaS(ColaS)


class PilaSAux:
    def __init__(self):
        self.top = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def apilarAux(self, data):
        nodo = Nodo(data)
        if self.isEmpty():
            self.top = nodo
        else:
            nodo.next = self.top
            self.top = nodo
        self.size += 1
        return data

    def desenpilAux(self):
        if self.isEmpty():
            print("Pila vacía")
            return None
        elim = self.top.data
        self.top = self.top.next
        self.size -= 1
        return elim

    def captarColaD(self, ColaDob):
        if ColaDob.isEmpty():
            return
        else:
          self.apilarAux(ColaDob.desenColarD())
          self.captarColaD(ColaDob)

    def imprimirPilaAux(self):
        valor = self.top
        print("Pila Aux: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")

In [24]:
cola = ColaDob()
cola.enColarD(1)
cola.enColarD(2)
cola.enColarD(3)

print('//////////////////')
cola.imprimirColaD()
print('//////////////////')
cola.imprimirColaIz()
print('//////////////////')
cola.desenColarD()
cola.imprimirColaD()


//////////////////
Cola: 1 => 2 => 3 => None
//////////////////
Cola: 3 => 2 => 1 => None
//////////////////
Cola: 2 => 3 => None


aca abajo la pila auxiliar que se podria usar para la cola 

In [ ]:
class PilaSAux:
    def __init__(self):
        self.top = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def apilarAux(self, data):
        nodo = Nodo(data)
        if self.isEmpty():
            self.top = nodo
        else:
            nodo.next = self.top
            self.top = nodo
        self.size += 1
        return data

    def desenpilAux(self):
        if self.isEmpty():
            print("Pila vacía")
            return None
        elim = self.top.data
        self.top = self.top.next
        self.size -= 1
        return elim

    def captarColaD(self, ColaDob):
        if ColaDob.isEmpty():
            return
        else:
          self.apilarAux(ColaDob.desenColarD())
          self.captarColaD(ColaDob)

    def imprimirPilaAux(self):
        valor = self.top
        print("Pila Aux: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")

//////////////////////////////////////////////////////////////////////////////////////////////////////////
Comienza lo buenos jsjsjjs

In [33]:

Cola = ColaDob()

with open('inventario1.txt', 'r') as file:
    for line in file:
        data = json.loads(line.strip())
        cola.enColarD(data)

# Imprimir la cola
cola.imprimirColaD()

FileNotFoundError: [Errno 2] No such file or directory: 'inventario1.txt'

In [34]:
import os
print("Directorio actual:", os.getcwd())

Directorio actual: c:\Users\USER\Documents\proyect\alg\Algoritmos-Trabajo-Final\backend


In [38]:
import json

class Nodo:
    def __init__(self, data=None):
        self.next = None
        self.data = data
        self.bef = None

class ColaDob:
    def __init__(self, limit=9):
        self.limite = limit
        self.front = None
        self.rear = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def isFull(self):
        return self.size >= self.limite

    def enColarD(self, data):
        nodo = Nodo(data)
        if self.isFull():
            print("Cola llena")
            return

        if self.isEmpty():
            self.front = self.rear = nodo
        else:
            nodo.bef = self.rear
            self.rear.next = nodo
            self.rear = nodo
        self.size += 1
        return data

    def desenColarD(self):
        if self.isEmpty():
            print("Cola vacía")
            return None
        nodo = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        else:
            self.front.bef = None
        self.size -= 1
        return nodo.data

    def imprimirColaD(self):
        valor = self.front
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")

    def imprimirColaIz(self):
        valor = self.rear
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.bef
        print("None")

    def captarColaS(self, ColaS):
        if ColaS.isEmpty():
            return
        else:
            self.enColarD(ColaS.desenColar())
            self.captarColaS(ColaS)

# Crear una instancia de la cola doblemente enlazada
cola = ColaDob()

# Leer el archivo de texto y llenar la cola
try:
    with open('inventario1.txt', 'r') as file:
        for line in file:
            data = json.loads(line.strip())
            cola.enColarD(data)
except FileNotFoundError:
    print("El archivo 'inventario1.txt' no se encontró. Asegúrate de que el archivo esté en el mismo directorio que tu código.")

# Imprimir la cola
cola.imprimirColaD()

print('///////////////')
cola.imprimirColaIz()


Cola: {'id': 1, 'nombre': 'Manzana', 'tipo': 'perecedero', 'cantidad': 50} => {'id': 2, 'nombre': 'Arroz', 'tipo': 'no perecedero', 'cantidad': 100} => {'id': 3, 'nombre': 'Leche', 'tipo': 'perecedero', 'cantidad': 30} => {'id': 4, 'nombre': 'Lentejas', 'tipo': 'no perecedero', 'cantidad': 80} => {'id': 5, 'nombre': 'Pollo', 'tipo': 'perecedero', 'cantidad': 20} => {'id': 6, 'nombre': 'Pasta', 'tipo': 'no perecedero', 'cantidad': 60} => None
///////////////
Cola: {'id': 6, 'nombre': 'Pasta', 'tipo': 'no perecedero', 'cantidad': 60} => {'id': 5, 'nombre': 'Pollo', 'tipo': 'perecedero', 'cantidad': 20} => {'id': 4, 'nombre': 'Lentejas', 'tipo': 'no perecedero', 'cantidad': 80} => {'id': 3, 'nombre': 'Leche', 'tipo': 'perecedero', 'cantidad': 30} => {'id': 2, 'nombre': 'Arroz', 'tipo': 'no perecedero', 'cantidad': 100} => {'id': 1, 'nombre': 'Manzana', 'tipo': 'perecedero', 'cantidad': 50} => None
